<a href="https://colab.research.google.com/github/0xZee/finetune_llama3-1_unsloth/blob/main/process_dataset_to_unsloth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
#!pip install huggingface_hub


In [72]:
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')

In [73]:
from datasets import load_dataset, Dataset

# Load the original dataset
ds = load_dataset("ArtifactAI/arxiv-math-instruct-50k")

print('data loaded')
print(ds)

data loaded
DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 50488
    })
})


In [74]:
# Flatten and clean
ds_flat = ds.flatten()
#ds_flat = ds_flat['train']
print('data flattened')
print(ds_flat)

data flattened
DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 50488
    })
})


In [75]:
# Rename columns
ds_newcol = ds_flat.rename_columns({"question": "instruction", "answer": "output"})
ds_newcol

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output'],
        num_rows: 50488
    })
})

In [76]:
# Add 'input' columns
ds_newcol_final = ds_newcol.map(lambda example: {"input": ""})
ds_newcol_final

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'input'],
        num_rows: 50488
    })
})

In [77]:
# Reorder the columns
ds_newcol_final = ds_newcol_final.select_columns(["instruction", "input", "output"])
ds_newcol_final

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 50488
    })
})

In [89]:
ds_newcol_final['train'][42]

{'instruction': 'What is a pad detector?',
 'input': '',
 'output': 'A pad detector is an electronic device used to detect the presence or absence of a conductive pad or material, typically used in industrial or automotive applications. It works by measuring the resistance or capacitance of the pad or material and producing an output signal based on the changes detected. Pad detectors are often used in applications such as brake pad wear monitoring, seat occupancy detection, and touch-sensitive control panels.'}

In [86]:
# Upload the processed dataset to the 0xZee Hugging Face user repository
DATASET_NAME = "0xZee/arxiv-math-Unsloth-tune-50k"
ds_newcol_final.push_to_hub(DATASET_NAME, token = userdata.get('HF_TOKEN') , commit_message='Arxiv Math Unsloth processed format', commit_description='3 columns Unsloth Format',)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/51 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/0xZee/arxiv-math-Unsloth-tune-50k/commit/e4650bb326493cfa7d286a6ce9a5edcb6c9c4afb', commit_message='Arxiv Math Unsloth processed format', commit_description='3 columns Unsloth Format', oid='e4650bb326493cfa7d286a6ce9a5edcb6c9c4afb', pr_url=None, pr_revision=None, pr_num=None)